In [1]:
import pandas as pd
import numpy as np
from sktime.forecasting.arima import ARIMA
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_squared_error
from sktime.utils.plotting import plot_series

import warnings

warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append("../utils/")

In [3]:
from sktime_custom_reduce import make_reduction
from lightgbm import LGBMRegressor
from sktime_custom_pipeline import ForecastingPipeline, TransformedTargetForecaster
from pipeline_helpers import get_fold_predictions, get_plotting_df, get_aeso_predictions, generate_step_predictions, generate_step_errors

### Pipeline to evaluate the default hyperparameters on the test set

In [4]:
X_train = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/train/X_train.csv",
    parse_dates=["date"],
    index_col="date",
)

y_train = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/train/y_train.csv",
    parse_dates=["date"],
    index_col="date",
)

X_train = X_train.sort_values(by="date")
X_train = X_train.asfreq("H")
y_train = y_train.sort_values(by="date")
y_train = y_train.asfreq("H")

In [5]:
X = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/test/X_test.csv",
    parse_dates=["date"],
    index_col="date",
)

y = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/test/y_test.csv",
    parse_dates=["date"],
    index_col="date",
)

X = X.sort_values(by="date")
X = X.asfreq("H")
y = y.sort_values(by="date")
y = y.asfreq("H")

In [6]:
y_test_full = y.copy()
X_test = X.copy()

forecast_len = 12
y_test = y_test_full[:-forecast_len]

In [7]:
def initialize_lgbm_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        (
                            "forecast",
                            make_reduction(
                                LGBMRegressor(
                                    boosting_type="dart",
                                    device="gpu",
                                    learning_rate=0.01,
                                    max_depth=15,
                                    min_data_in_leaf=20,
                                    n_estimators=1000,
                                    num_leaves=70,
                                    num_threads=13,
                                    reg_alpha=30,
                                    reg_lambda=20,
                                ),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe

lgbm_pipeline = initialize_lgbm_forecaster()

fh = ForecastingHorizon(np.arange(1, 12 + 1))

In [8]:
rolling_prediction_df = pd.DataFrame(index=y_test_full.index)

In [9]:
lgbm_pipeline.fit(y=y_train, X=X_train, fh=fh)

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] num_threads is set=13, n_jobs=-1 will be ignored. Current value: num_threads=13


ForecastingPipeline(steps=[('forecaster',
                            TransformedTargetForecaster(steps=[('forecast',
                                                                DirectTabularRegressionForecaster(estimator=LGBMRegressor(boosting_type='dart', device='gpu', learning_rate=0.01,
              max_depth=15, min_data_in_leaf=20, n_estimators=1000,
              num_leaves=70, num_threads=13, reg_alpha=30, reg_lambda=20),
                                                                                                  window_length=24))]))])

In [10]:
y_pred = lgbm_pipeline.predict(fh, X=X_train.tail(1))
y_pred.columns = [f"cutoff_hour_{lgbm_pipeline.cutoff.hour[0]}"]
rolling_prediction_df = pd.concat([rolling_prediction_df, y_pred], axis=1)

In [11]:
rolling_prediction_df

,cutoff_hour_23
2023-02-01 00:00:00,74.331564
2023-02-01 01:00:00,71.878880
2023-02-01 02:00:00,67.493641
2023-02-01 03:00:00,69.973898
2023-02-01 04:00:00,66.480472
...,...
2023-05-30 20:00:00,NaN
2023-05-30 21:00:00,NaN
2023-05-30 22:00:00,NaN
2023-05-30 23:00:00,NaN


In [12]:
# emulating the rolling prediction for the next hours

for i in range(0, len(y_test), 12):

        new_observation_y, new_observation_X  = y_test_full[i:i+12], X_test[i:i+12]
        
        new_observation_y = new_observation_y.asfreq('H')
        new_observation_X = new_observation_X.asfreq('H')

        print(f'Updating with actual values at {new_observation_y.index[0]}')
        print(f'Cut off before update: {lgbm_pipeline.cutoff}')

        lgbm_pipeline.update(y=new_observation_y, X=new_observation_X, update_params=True)

        print(f'Cut off after update: {lgbm_pipeline.cutoff}')

        lgbm_pipeline.cutoff.freq = 'H'

        cutoff_time = lgbm_pipeline.cutoff
        prediction_for = cutoff_time + pd.DateOffset(hours=i)

        print(f'Predicting for {prediction_for}')
        
        y_pred = lgbm_pipeline.predict(fh, X=new_observation_X)
        
        y_pred.columns = [f"cutoff_hour_{lgbm_pipeline.cutoff.hour[0]}"]
        
        rolling_prediction_df = pd.concat([rolling_prediction_df, y_pred], axis=1)
        
        print(f'Update and prediction done for {new_observation_y.index[0]}')
        print(f'----------------------------------------------------------------------------------')

Updating with actual values at 2023-02-01 00:00:00
Cut off before update: DatetimeIndex(['2023-01-31 23:00:00'], dtype='datetime64[ns]', name='date', freq='H')
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] num_threads is set=13, n_jobs=-1 will be ignored. Current value: num_threads=13
Cut off after update: DatetimeIndex(['2023-02-01 11:00:00'], dtype='datetime64[ns]', name='date', freq='H')
Predicting for DatetimeIndex(['2023-02-01 11:00:00'], dtype='datetime64[ns]', name='date', freq=None)
Update and prediction done for 2023-02-01 00:00:00
----------------------------------------------------------------------------------
Updating with actual values at 2023-02-01 12:00:00
Cut off before update: DatetimeIndex(['2023-02-01 11:00:00'], dtype='datetime64[ns]', name='date', freq='H')
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in

In [13]:
rolling_prediction_df

,cutoff_hour_23,cutoff_hour_11,cutoff_hour_23,cutoff_hour_11,cutoff_hour_23,cutoff_hour_11,cutoff_hour_23,cutoff_hour_11,cutoff_hour_23,cutoff_hour_11,...,cutoff_hour_11,cutoff_hour_23,cutoff_hour_11,cutoff_hour_23,cutoff_hour_11,cutoff_hour_23,cutoff_hour_11,cutoff_hour_23,cutoff_hour_11,cutoff_hour_23
2023-02-01 00:00:00,74.331564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 01:00:00,71.878880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 02:00:00,67.493641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 03:00:00,69.973898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 04:00:00,66.480472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.229923
2023-05-31 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.955927
2023-05-31 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.307162
2023-05-31 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.730557


In [14]:
fold_actuals, fold_predictions_list, rmse_list = get_fold_predictions(rolling_prediction_df, y_test_full)

In [15]:
# Print Average RMSE of all folds
print(f"Average RMSE for each fold: {np.mean(rmse_list)}")

Average RMSE for each fold: 133.9004409933885


In [16]:
# Print max RMSE
print(f"Max RMSE for each fold: {np.max(rmse_list)}")

Max RMSE for each fold: 587.3576702380615


In [17]:
# Print hightst 5 RMSE
print(f"Top 5 RMSE for each fold: {np.sort(rmse_list)[-5:]}")

Top 5 RMSE for each fold: [457.57011049 513.44834829 527.56446809 539.49326683 587.35767024]


In [18]:
y_hist = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/filtered_target_medium.csv",
    parse_dates=["date"],
    index_col="date",
)

y_hist = y_hist.sort_values(by="date")
y_hist = y_hist.asfreq("H")

In [19]:
ddf = get_plotting_df(fold_actuals=fold_actuals, fold_predictions_list=fold_predictions_list, y_hist=y_hist)

In [20]:
import plotly.express as px
fig = px.line(ddf, x="periodstep", y=["HistoricalPrice", "FuturePrice", "Predicted"], animation_frame="timestep")
fig.show()

In [21]:
aeso_predictions_df = get_aeso_predictions(y_test_full.index[0], y_test_full.index[-1])
rmse_aeso_predictions = mean_squared_error(aeso_predictions_df['actual'], aeso_predictions_df['forecast'], squared=False)
print(f"RMSE for the predictions by AESO for the same time period as the test set: {round(rmse_aeso_predictions, 2)} CAD/MWh")

RMSE for the predictions by AESO for the same time period as the test set: 118.55 CAD/MWh
